In [4]:
#!/usr/bin/env python
# coding: utf-8

import sys

from numpy import (
    fromiter,
    multiply
)

from pandas import (
    read_csv,
    concat,
    DataFrame
)

from string import (
    ascii_uppercase
)

# from typing import (
#     Tuple
# )


def input_importer(
        cfps_parameters,
        initial_set_concentrations,
        normalizer_set_concentrations,
        autofluorescence_set_concentrations):
    """
    Create pandas dataframes from tsv files.

    Parameters
    ----------
    cfps_parameters : tsv file
        Tsv with list of CFPS parameters and relative features.
    initial_set_concentrations : tsv file
        Initial training set with concentrations values.
    normalizer_set_concentrations : tsv file
        Normalizer set with concentrations values.
    autofluorescence_set_concentrations : tsv file
        Autofluorescence set with concentrations values.

    Returns
    -------
    cfps_parameters_df : DataFrame
        Pandas dataframe populated with cfps_parameters data.
    initial_set_volumes_df : DataFrame
        Pandas dataframe with initial_set_concentrations data.
    normalizer_set_volumes_df : DataFrame
        Pandas dataframe with normalizer_set_concentrations data.
    autofluorescence_set_volumes_df : DataFrame
        Pandas dataframe with autofluorescence_set_concentrations data.
    """
    cfps_parameters_df = read_csv(
        cfps_parameters,
        sep='\t')

    initial_set_concentrations_df = read_csv(
        initial_set_concentrations,
        sep='\t')

    normalizer_set_concentrations_df = read_csv(
        normalizer_set_concentrations,
        sep='\t')

    autofluorescence_set_concentrations_df = read_csv(
        autofluorescence_set_concentrations,
        sep='\t')

    return (cfps_parameters_df,
            initial_set_concentrations_df,
            normalizer_set_concentrations_df,
            autofluorescence_set_concentrations_df)


def volumes_array_generator(
        cfps_parameters_df,
        initial_set_concentrations_df,
        normalizer_set_concentrations_df,
        autofluorescence_set_concentrations_df,
        sample_volume):
    """
    Convert concentrations dataframes into volumes dataframes.

    Parameters
    ----------
    cfps_parameters_df : DataFrame
        Pandas dataframe populated with cfps_parameters data.
    initial_set_volumes_df : DataFrame
        Pandas dataframe with initial_set_concentrations data.
    normalizer_set_volumes_df : DataFrame
        Pandas dataframe with normalizer_set_concentrations data.
    autofluorescence_set_volumes_df : DataFrame
        Pandas dataframe with autofluorescence_set_concentrations data.
    sample_volume: int
        Final sample volume in each well.

    Returns
    -------
    initial_set_volumes_df : DataFrame
        Initial set with volumes values.
    normalizer_set_volumes_df : DataFrame
        Normalizer set with volumes values.
    autofluorescence_set_volumes_df : DataFrame
        Autofluorescence set with volumes values.
    """
    stock_concentrations_dict = dict(
        cfps_parameters_df[['Parameter', 'Stock concentration']].to_numpy())

    stock_concentrations_array = fromiter(
        stock_concentrations_dict.values(),
        dtype=float)

    stock_concentrations_array = \
        sample_volume / \
        stock_concentrations_array / \
        0.0025

    intial_set_volumes_df = (multiply(
        initial_set_concentrations_df,
        stock_concentrations_array)) * 0.0025

    normalizer_set_volumes_df = (multiply(
        normalizer_set_concentrations_df,
        stock_concentrations_array)) * 0.0025

    autofluorescence_set_volumes_df = (multiply(
        autofluorescence_set_concentrations_df,
        stock_concentrations_array)) * 0.0025

    return (intial_set_volumes_df,
            normalizer_set_volumes_df,
            autofluorescence_set_volumes_df)


def save_volumes_array(
        cfps_parameters_df,
        intial_set_volumes_df,
        normalizer_set_volumes_df,
        autofluorescence_set_volumes_df):
    """
    Save Pandas dataframes in tsv files.

    Parameters
    ----------
    cfps_parameters_df : DataFrame
        Pandas dataframe populated with cfps_parameters data.
    initial_set_volumes_df : DataFrame
        Initial set with volumes values.
    normalizer_set_volumes_df : DataFrame
        Normalizer set with volumes values.
    autofluorescence_set_volumes_df : DataFrame
        Autofluorescence set with volumes values.

    Returns
    -------
    initial_set_volumes : tsv file
        Initial set with volumes values.
    normalizer_set_volumes : tsv file
        Normalizer set with volumes values.
    autofluorescence_set_volumes : tsv file
        Autofluorescence set with volumes values.
    """
    all_parameters = cfps_parameters_df['Parameter'].tolist()

    initial_set_volumes = intial_set_volumes_df.to_csv(
        '/Users/yorgo/Desktop/INRAe/icfree-ml/data/volumes_output/intial_set_volumes.tsv',
        sep='\t',
        header=all_parameters,
        index=False)

    normalizer_set_volumes = normalizer_set_volumes_df.to_csv(
        '/Users/yorgo/Desktop/INRAe/icfree-ml/data/volumes_output/normalizer_set_volumes.tsv',
        sep='\t',
        header=all_parameters)

    autofluorescence_set_volumes = autofluorescence_set_volumes_df.to_csv(
        '/Users/yorgo/Desktop/INRAe/icfree-ml/data/volumes_output/autofluorescence_set_volumes.tsv',
        sep='\t',
        header=all_parameters,
        index=False)

    return (initial_set_volumes,
            normalizer_set_volumes,
            autofluorescence_set_volumes)


def destination_plate_generator(
        initial_set_volumes_df,
        normalizer_set_volumes_df,
        autofluorescence_set_volumes_df,
        starting_well,
        vertical):
    """
    Make a dataframe as a 384 well plate for each metabolite.

    Parameters
    ----------
    volumes_array : DataFrame
        _description_
    starting_well : str
        Name of the starter well to begin filling the 384 well-plate.
    vertical: bool
        -True: plate is filled column by column from top to bottom.
        -False: plate is filled row by row from left to right.

    Returns
    -------
    all_dataframe:
        _description_

    volumes_wells: DataFrame
        _description_
    """
    all_dataframe = {}
    volumes_array_list = [
        initial_set_volumes_df,
        normalizer_set_volumes_df,
        autofluorescence_set_volumes_df]
    destination_plates_list = []
    plate_rows = ascii_uppercase
    plate_rows = list(plate_rows[0:16])

    for volumes_array in volumes_array_list:
        if vertical:
            from_well = plate_rows.index(starting_well[0]) + \
                (int(starting_well[1:]) - 1) * 16

            for parameter_name in volumes_array.columns:
                dataframe = DataFrame(
                    0.0,
                    index=plate_rows,
                    columns=range(1, 25))

                for index, value in enumerate(volumes_array[parameter_name]):
                    index += from_well
                    dataframe.iloc[index % 16, index // 16] = value

                all_dataframe[parameter_name] = dataframe

            volumes_wells = volumes_array.copy()
            names = ['{}{}'.format(
                plate_rows[(index + from_well) % 16],
                (index + from_well) // 16 + 1)
                    for index in volumes_wells.index]

            volumes_wells['well_name'] = names
            destination_plates_list.append(volumes_wells)

        if not vertical:
            from_well = plate_rows.index(starting_well[0]) * 24 + \
                int(starting_well[1:]) - 1

            for parameter_name in volumes_array.columns:
                dataframe = DataFrame(
                    0.0,
                    index=plate_rows,
                    columns=range(1, 25))

                for index, value in enumerate(volumes_array[parameter_name]):
                    index += from_well
                    dataframe.iloc[index // 24, index % 24] = value

                all_dataframe[parameter_name] = dataframe

            volumes_wells = volumes_array.copy()
            names = ['{}{}'.format(
                plate_rows[index // 24],
                index % 24 + 1) for index in volumes_wells.index]
            volumes_wells['well_name'] = names
            destination_plates_list.append(volumes_wells)

    return all_dataframe, destination_plates_list


def echo_instructions_generator(destination_plates_list):
    """
    Generate instructions matrix for the Echo robot

    Parameters
    ----------
        volumes_wells: _type_
            _description_

    Returns
    -------
        all_sources: _type_
            _description_
        echo_instructions: _type_
            _description_
    """
    all_sources = {}
    echo_instructions_list = []

    for destination_plate in destination_plates_list:

        for parameter_name in destination_plate.drop(columns=['well_name']):
            transfers = {
                'Source_Plate_Barcode': [],
                'Source_Well': [],
                'Destination_Plate_Barcode': [],
                'Destination_Well': [],
                'Transfer_Volume': []}

            for index in range(len(destination_plate)):
                if destination_plate.loc[index, parameter_name] > 0:
                    transfers['Source_Plate_Barcode'].append('Plate1')
                    transfers['Source_Well'].append(
                        '{} well'.format(parameter_name))
                    transfers['Destination_Plate_Barcode'].append('destPlate1')
                    transfers['Destination_Well'].append(
                        destination_plate.loc[index, 'well_name'])
                    transfers['Transfer_Volume'].append(
                        destination_plate.loc[index, parameter_name])
            transfers = DataFrame(transfers)
            all_sources[parameter_name] = transfers

        echo_instructions = concat(all_sources.values())
        echo_instructions_list.append(echo_instructions)

    return echo_instructions_list


def save_echo_instructions(echo_instructions_list):
    """
    Save instructions matrix in a tsv file

    Parameters
    ----------
        echo_instructions: _type_
            _description_
    """

    for echo_instructions in echo_instructions_list:
        echo_instructions.to_csv(
            '/Users/yorgo/Desktop/INRAe/icfree-ml/data/echo_instructions/echo_instructions.tsv',
            sep='\t',
            index=False)


In [5]:
if __name__ == "__main__":
    
    cfps_parameters = '/Users/yorgo/Desktop/INRAe/icfree-ml/data/initial_input/proCFPS_parameters.tsv'
    initial_set_concentrations = '/Users/yorgo/Desktop/INRAe/icfree-ml/data/initial_output/initial_training_set.tsv' 
    normalizer_set_concentrations = '/Users/yorgo/Desktop/INRAe/icfree-ml/data/initial_output/normalizer_set.tsv'
    autofluorescence_set_concentrations = '/Users/yorgo/Desktop/INRAe/icfree-ml/data/initial_output/autofluorescence_set.tsv'
    sample_volume = 10

    cfps_parameters_df = input_importer(
        cfps_parameters,
        initial_set_concentrations,
        normalizer_set_concentrations,
        autofluorescence_set_concentrations)[0]

    initial_set_concentrations_df = input_importer(
        cfps_parameters,
        initial_set_concentrations,
        normalizer_set_concentrations,
        autofluorescence_set_concentrations)[1]

    normalizer_set_concentrations_df = input_importer(
        cfps_parameters,
        initial_set_concentrations,
        normalizer_set_concentrations,
        autofluorescence_set_concentrations)[2]

    autofluorescence_set_concentrations_df = input_importer(
        cfps_parameters,
        initial_set_concentrations,
        normalizer_set_concentrations,
        autofluorescence_set_concentrations)[3]

    initial_set_volumes_df = volumes_array_generator(
        cfps_parameters_df,
        initial_set_concentrations_df,
        normalizer_set_concentrations_df,
        autofluorescence_set_concentrations_df,
        sample_volume)[0]

    normalizer_set_volumes_df = volumes_array_generator(
        cfps_parameters_df,
        initial_set_concentrations_df,
        normalizer_set_concentrations_df,
        autofluorescence_set_concentrations_df,
        sample_volume)[1]

    autofluorescence_set_volumes_df = volumes_array_generator(
        cfps_parameters_df,
        initial_set_concentrations_df,
        normalizer_set_concentrations_df,
        autofluorescence_set_concentrations_df,
        sample_volume)[2]

    save_volumes_array(
        cfps_parameters_df,
        initial_set_volumes_df,
        normalizer_set_volumes_df,
        autofluorescence_set_volumes_df)

    destination_plates_list = destination_plate_generator(
        initial_set_volumes_df,
        normalizer_set_volumes_df,
        autofluorescence_set_volumes_df,
        starting_well='A1',
        vertical=True)[1]

    echo_instructions_list = echo_instructions_generator(
        destination_plates_list)

    save_echo_instructions(echo_instructions_list)

    print(destination_plates_list)


[    Mg-glutamate  k-glutamate  Sodium oxalate     3-PGA       NTP       NAD  \
0       0.000000     0.845238            0.02  0.128571  0.019048  1.298701   
1       0.495238     0.845238            0.00  0.171429  0.019048  0.000000   
2       0.495238     0.000000            0.10  0.042857  0.023810  2.164502   
3       0.123810     0.211310            0.00  0.042857  0.014286  1.731602   
4       0.619048     0.000000            0.06  0.128571  0.023810  0.000000   
..           ...          ...             ...       ...       ...       ...   
95      0.000000     0.000000            0.08  0.042857  0.019048  0.000000   
96      0.123810     0.633929            0.08  0.214286  0.019048  0.000000   
97      0.123810     0.633929            0.00  0.042857  0.019048  0.432900   
98      0.123810     1.056548            0.00  0.128571  0.014286  0.432900   
99      0.495238     1.056548            0.08  0.000000  0.000000  1.298701   

    Folinic acid  Spermidine      tRNA  Amino acid